In [1]:
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import pandas as pd
np.set_printoptions(precision=2, linewidth=120, suppress=True, edgeitems=4)

In [2]:
agg_set = pd.read_csv('daily_hood_type.csv',sep=",",quotechar='"')

In [3]:
# storing the community id to community name in a dictionary
comm_dict = {}
i =0 
with open('CommAreas.csv','r') as infl:
    for line in infl.readlines():
        if i==0:
            i+=1
            continue
        comlst = line.split('"')
        comm_dict[comlst[2].split(',')[5]]=comlst[2].split(',')[6]
        i+=1
type_dict={'Other':0,'Property':1,'Violent':2}
        

In [4]:
ndays = 1047
ncomm = len(comm_dict)
nrows = ndays*ncomm
ncols = 2*2*3
crime_mat = np.zeros((nrows,ncols))
print agg_set.head()
print agg_set.shape
print crime_mat.shape

   Day  Community      Type Arrest Domestic  Total
0    0         11     Other  False     True      1
1    0         14  Property  False    False      1
2    0         16     Other  False    False      1
3    0         17  Property  False    False      2
4    0         22     Other  False    False      1
(308981, 6)
(80619L, 12L)


In [5]:
i=0
while i < len(agg_set):
    row = (agg_set['Community'][i]-1)*(ndays)
#    print row
    row += agg_set['Day'][i]
#    print beat_agg_set.index[i][0]
    ctype = type_dict[agg_set['Type'][i]]
    arr = agg_set['Arrest'][i]
    dom = agg_set['Domestic'][i]
    col = ctype*4
#    print col
    if not arr:
        col+=2
    if not dom:
        col+=1
    crime_mat[row,col] = agg_set['Total'][i]
    i+=1
print crime_mat.shape
print crime_mat

(80619L, 12L)
[[ 0.  0.  0.  0. ...,  0.  0.  0.  0.]
 [ 0.  1.  0.  4. ...,  1.  0.  1.  0.]
 [ 0.  0.  1.  1. ...,  0.  0.  0.  2.]
 [ 0.  0.  0.  2. ...,  0.  0.  0.  1.]
 ..., 
 [ 0.  0.  0.  1. ...,  0.  0.  2.  0.]
 [ 0.  0.  0.  1. ...,  0.  1.  0.  1.]
 [ 0.  0.  0.  1. ...,  0.  0.  0.  0.]
 [ 0.  0.  0.  0. ...,  0.  0.  1.  0.]]


In [6]:
# counting empty rows
z=[]
for i in range(crime_mat.shape[0]):
    if sum(crime_mat[i])==0:
        z.append(i)
print len(z)


3652


In [7]:
# normalizing
print crime_mat
crime_norm = np.zeros(crime_mat.shape)
i = 0
for m in crime_mat:
    n = np.linalg.norm(m)
    if n!=0:
        crime_norm[i]= m / n
    i+=1
print crime_norm
z=[]
for i in range(crime_mat.shape[0]):
    if sum(crime_mat[i])==0:
        z.append(i)
print len(z)


[[ 0.  0.  0.  0. ...,  0.  0.  0.  0.]
 [ 0.  1.  0.  4. ...,  1.  0.  1.  0.]
 [ 0.  0.  1.  1. ...,  0.  0.  0.  2.]
 [ 0.  0.  0.  2. ...,  0.  0.  0.  1.]
 ..., 
 [ 0.  0.  0.  1. ...,  0.  0.  2.  0.]
 [ 0.  0.  0.  1. ...,  0.  1.  0.  1.]
 [ 0.  0.  0.  1. ...,  0.  0.  0.  0.]
 [ 0.  0.  0.  0. ...,  0.  0.  1.  0.]]
[[ 0.    0.    0.    0.   ...,  0.    0.    0.    0.  ]
 [ 0.    0.13  0.    0.53 ...,  0.13  0.    0.13  0.  ]
 [ 0.    0.    0.25  0.25 ...,  0.    0.    0.    0.5 ]
 [ 0.    0.    0.    0.53 ...,  0.    0.    0.    0.27]
 ..., 
 [ 0.    0.    0.    0.45 ...,  0.    0.    0.89  0.  ]
 [ 0.    0.    0.    0.19 ...,  0.    0.19  0.    0.19]
 [ 0.    0.    0.    0.41 ...,  0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.   ...,  0.    0.    0.58  0.  ]]
3652


In [8]:
# t will hold the community labels for comparison with the clustering results
t=[]
for i in range(crime_mat.shape[0]):
    t.append((int(i/ndays)+1))

In [9]:
# save the matrices
with open('crime_mat.csv','w') as outf:
    for i in range(crime_mat.shape[0]):
        outf.write(str(t[i]))
        for j in range(0,crime_mat.shape[1]):
            outf.write(','+str(crime_mat[i,j]))
        outf.write('\n')
with open('crime_norm.csv','w') as outf:
    for i in range(crime_norm.shape[0]):
        outf.write(str(t[i]))
        for j in range(0,crime_norm.shape[1]):
            outf.write(','+str(crime_norm[i,j]))
        outf.write('\n')

In [10]:
from sklearn.cluster import KMeans
from sklearn.metrics import completeness_score, homogeneity_score

In [11]:
ss=dt.datetime.now()
complt_e=[]
homoge_e=[]
for k in range(1,20):
    mx_c=[]
    mx_h=[]
    sss=dt.datetime.now()
    for i in range(10):
        kclus=KMeans(n_clusters=k).fit(crime_mat)
        c = kclus.labels_
        hom = homogeneity_score(t,c)
        com = completeness_score(t,c)
        mx_h.append(hom)
        mx_c.append(com)
        #if hom > mx_h:
        #    mx_h = hom
        #if com > mx_c:
        #    mx_c = com
    mx_h=np.mean(mx_h)
    mx_c=np.mean(mx_c)
    homoge_e.append(mx_h)
    complt_e.append(mx_c)
    print k,mx_c,mx_h, dt.datetime.now()-sss
print dt.datetime.now()-ss

1 1.0 2.46003114188e-16 0:00:01.577000
2 0.525648255013 0.0650664615921 0:00:08.915000
3 0.487729082793 0.0981611636792 0:00:23.420000
4 0.465009136986 0.108633835896 0:00:34.169000
5 0.4633145674 0.123561719393 0:00:37.500000
6 0.420059912913 0.131959384602 0:00:47.985000
7 0.398634284836 0.134335183086 0:01:00.137000
8 0.363341416012 0.139836835293 0:01:11.925000
9 0.35976833588 0.148442449975 0:01:14.898000
10 0.345996722249 0.150586452852 0:01:28.947000
11 0.336403671061 0.15348443251 0:01:58.945000
12 0.327729719445 0.155496927721 0:02:03.849000
13 0.321168670326 0.157147005512 0:02:23.670000
14 0.31248245726 0.158459252264 0:02:37.741000
15 0.309131194964 0.159564923841 0:03:05.603000
16 0.305593801552 0.160635128582 0:03:05.197000
17 0.302628280544 0.162324251242 0:04:03.388000
18 0.297230448157 0.163484379692 0:03:45.345000
19 0.293285803669 0.164695417805 0:03:55.185000
0:34:28.404000


In [12]:
ss=dt.datetime.now()
complt_c=[]
homoge_c=[]
for k in range(1,20):
    mx_c=[]
    mx_h=[]
    sss=dt.datetime.now()
    for i in range(10):
        kclus=KMeans(n_clusters=k).fit(crime_norm)
        c = kclus.labels_
        hom = homogeneity_score(t,c)
        com = completeness_score(t,c)
        mx_h.append(hom)
        mx_c.append(com)
        #if hom > mx_h:
        #    mx_h = hom
        #if com > mx_c:
        #    mx_c = com
    mx_h=np.mean(mx_h)
    mx_c=np.mean(mx_c)
    homoge_c.append(mx_h)
    complt_c.append(mx_c)
    print k,mx_c,mx_h, dt.datetime.now()-sss
print dt.datetime.now()-ss

1 1.0 2.46003114188e-16 0:00:02.054000
2 0.115140149817 0.0176752591623 0:00:09.914000
3 0.142326273257 0.0327257665185 0:00:18.824000
4 0.152191487028 0.043290333667 0:00:23.699000
5 0.146291069804 0.0470557609286 0:00:26.041000
6 0.137119852947 0.0491023140936 0:00:28.069000
7 0.135584495166 0.0547953721726 0:00:36.266000
8 0.129410182958 0.0573881756317 0:00:42.034000
9 0.126248635757 0.0595560990788 0:00:50.248000
10 0.125692339425 0.0617079943415 0:01:00.791000
11 0.128142720412 0.0657579619849 0:01:04.716000
12 0.128903149958 0.0686213913936 0:01:08.532000
13 0.128404716226 0.0704503562971 0:01:20.720000
14 0.12904888077 0.0732442539695 0:01:22.486000
15 0.128886073315 0.0746543856933 0:01:29.036000
16 0.127471243241 0.0758005896536 0:01:35.096000
17 0.128393091443 0.0781615096331 0:01:42.426000
18 0.126934928991 0.0793041361781 0:01:51.739000
19 0.126987802353 0.0808305491861 0:01:53.616000
0:18:26.313000


In [13]:
xaxis=range(1,20)
plt.figure()
plt.plot(xaxis,homoge_c,'orange',label='Normed\nDistance\nHomogeneity')
plt.plot(xaxis,complt_c,'green',label='Normed\nDistance\nCompleteness')
plt.plot(xaxis,homoge_e,'red',label='Unnormed\nDistance\nHomogeneity')
plt.plot(xaxis,complt_e,'blue',label='Unnormed\nDistance\nCompleteness')
plt.plot(xaxis,homoge_c,'.',color='orange')
plt.plot(xaxis,complt_c,'.',color='green')
plt.plot(xaxis,homoge_e,'.',color='red')
plt.plot(xaxis,complt_e,'.',color='blue')
plt.title('Homogeneity and Completeness\nBetween Cluster and Beat')
plt.xlabel('Number of clusters')
plt.ylabel('Score')
plt.legend()
plt.show()

In [14]:
def clus_stats(clus,K):
    clus_count = [] # size of cluster
    for k in range(K):
        clus_count.append(sum(clus.labels_==k)) # sum total entries in cluster
    return(clus_count)

In [15]:
K=5
kclus=KMeans(n_clusters=K).fit(crime_mat)
print clus_stats(kclus,K)
for c in kclus.cluster_centers_:
    print np.linalg.norm(c)
    print c

[45411, 6117, 1066, 19363, 8662]
1.60591925307
[ 0.02  0.38  0.1   0.49  0.01  0.17  0.07  1.28  0.11  0.17  0.35  0.6 ]
12.984759469
[  0.05   1.5    0.29   2.41   0.03   1.09   0.28  12.14   0.35   0.82   1.31   3.06]
18.2678787445
[  0.16  11.69   0.81   3.22   0.15   1.28   0.97  10.58   1.41   1.52   4.58   6.79]
5.63601044212
[ 0.03  0.74  0.2   1.05  0.02  0.41  0.18  5.18  0.27  0.41  0.82  1.46]
7.55987104894
[ 0.06  4.18  0.35  1.31  0.06  0.45  0.47  4.43  0.56  0.73  2.23  3.46]


In [16]:
for c in kclus.cluster_centers_:
    print sum(c)
    for i in np.argsort(c)[::-1][:5]:
        r=i%4
        if r==0:
            s='a:t d:t'
        elif r==1:
            s='a:t d:f'
        elif r==2:
            s='a:f d:t'
        else:
            s='a:f d:f'
        r=int(i/4)
        if r==0:
            p='Other'
        if r==1:
            p='Property'
        else:
            p='Violent'
        print p, s,c[i]
    print

3.73612823674
Property a:f d:f 1.27607010745
Violent a:f d:f 0.602408842699
Violent a:f d:f 0.487735599788
Violent a:t d:f 0.375264224062
Violent a:f d:t 0.354412541836

23.3302812296
Property a:f d:f 12.137998692
Violent a:f d:f 3.05673642904
Violent a:f d:f 2.41219751472
Violent a:t d:f 1.50098103336
Violent a:f d:t 1.30591890124

43.1480787254
Violent a:t d:f 11.694470478
Property a:f d:f 10.5810684161
Violent a:f d:f 6.79475164011
Violent a:f d:t 4.57544517338
Violent a:f d:f 3.21649484536

10.7756936909
Property a:f d:f 5.17831860693
Violent a:f d:f 1.46411409084
Violent a:f d:f 1.04743450628
Violent a:f d:t 0.822921510877
Violent a:t d:f 0.744794088772

18.2796815507
Property a:f d:f 4.42909888081
Violent a:t d:f 4.18206992039
Violent a:f d:f 3.45886696666
Violent a:f d:t 2.22729895004
Violent a:f d:f 1.31498788508



In [17]:
assign=kclus.labels_
clus_count = np.zeros(5)
comm_clus = []
with open('comm_clus.csv','w') as outf:
    i=0
    outf.write('comm,cluster,name\n')
    c_vote=None
    while i<len(assign):
        if i%ndays==0:
            if c_vote:
                c = int(max(set(c_vote),key=c_vote.count))
                outs = "%s,%s,%s\n"%(int(i/ndays)+1,c,comm_dict[str(int(i/ndays))])
                comm_clus.append(c)
#                print outs
                outf.write(outs)
                clus_count[c]+=1 
            c_vote=[assign[i]]
        else:
            c_vote.append(assign[i])
        i+=1
    c = int(max(set(c_vote),key=c_vote.count))
    outf.write("%s,%s,%s\n"%(int(i/ndays)+1,c,comm_dict[str(int(i/ndays))]))
    comm_clus.append(c)
print clus_count

[ 45.   4.   1.  16.  10.]


In [18]:
for c in comm_dict:print comm_clus[int(c)-1], comm_dict[c]

0 GARFIELD RIDGE
0 EDGEWATER
0 RIVERDALE
1 NEAR WEST SIDE
0 CALUMET HEIGHTS
4 SOUTH SHORE
0 BRIDGEPORT
3 NEW CITY
0 WEST ELSDON
0 GAGE PARK
0 CLEARING
4 ROSELAND
3 CHICAGO LAWN
4 WEST ENGLEWOOD
4 ENGLEWOOD
4 GREATER GRAND CROSSING
0 EAST SIDE
3 WEST PULLMAN
1 WEST TOWN
2 AUSTIN
4 WEST GARFIELD PARK
4 EAST GARFIELD PARK
0 HERMOSA
0 AVONDALE
3 LOGAN SQUARE
4 HUMBOLDT PARK
3 SOUTH CHICAGO
0 BURNSIDE
3 CHATHAM
0 AVALON PARK
3 WOODLAWN
4 NORTH LAWNDALE
0 WASHINGTON PARK
0 HYDE PARK
3 ROGERS PARK
3 UPTOWN
3 WEST RIDGE
0 NORTH CENTER
0 LINCOLN SQUARE
3 LINCOLN PARK
3 LAKE VIEW
0 EDISON PARK
1 NEAR NORTH SIDE
0 SOUTH DEERING
0 NORTH PARK
0 WEST LAWN
0 OHARE
0 MORGAN PARK
0 FOREST GLEN
0 WASHINGTON HEIGHTS
0 BEVERLY
4 AUBURN GRESHAM
0 ASHBURN
0 MCKINLEY PARK
0 BRIGHTON PARK
0 JEFFERSON PARK
0 NORWOOD PARK
0 KENWOOD
0 GRAND BOULEVARD
3 PORTAGE PARK
0 ALBANY PARK
0 DUNNING
3 IRVING PARK
3 BELMONT CRAGIN
0 MONTCLARE
0 LOWER WEST SIDE
3 SOUTH LAWNDALE
0 FULLER PARK
0 OAKLAND
0 DOUGLAS
0 ARMOUR SQUA